In [ ]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance
# !{sys.executable} -m pip install binance-connector
# !{sys.executable} -m pip install pandas

import os
from binance.spot import Spot

import pandas as pd
from dotenv import load_dotenv
pd.set_option("display.precision", 8)

In [2]:
# read from local .env file
load_dotenv() 

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']

spotClient = Spot()
# myClient = Client(api_key, api_secret, testnet=True)

In [3]:
# INPUT
## SET this to get d2, d3, d4
day_aggregation = 18

## ema (20 or 25)
ema_size = 25

## how many day to query
quantity = 3000

## trading pair
symbol = "ONTBTC"



In [4]:
# TIMEFRAME
## set timeframe to query binance, based on day_aggregation
do_manual_aggregation = True
timeframe = '1d'
# if day_aggregation == 3:
#     timeframe = '3d'
#     do_manual_aggregation = False
# elif day_aggregation == 7:
#     timeframe = '1w'
#     do_manual_aggregation = False
# elif day_aggregation == 30:
#     timeframe = '1M'
#     do_manual_aggregation = False
        

In [5]:
# FUNCTIONS
## Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = spotClient.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

## Define function to calculate an arbitrary EMA      
def calc_generic_ema(dataframe, symbol, timeframe, ema_size):
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [6]:
# MAIN
raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=quantity)
## Convert into Dataframe
df = pd.DataFrame(raw_data)

df['time'] = pd.to_datetime(df['time'], unit='ms')


if do_manual_aggregation:
    df['d_index'] = df.index//day_aggregation
    # get high / low
    df['d_high'] = df.groupby('d_index')['high'].transform(max)
    df['d_low'] = df.groupby(['d_index'])['low'].transform(min)
    
    # get open / close
    
    df['d_open'] = df.groupby(['d_index'])['open'].transform('first')
    df['d_close'] = df.groupby(['d_index'])['close'].transform('last')

    df = df.groupby('d_index').first()
    
    # clean up d1 column
    del df['open']
    del df['high']
    del df['low']
    del df['close']
    
    # rename d_column
    df = df.rename(columns={"d_high": "high", "d_low": "low", "d_close": "close", "d_open": "open" })
    
# ema
df = calc_generic_ema(df, symbol, timeframe, ema_size)

# clean up

del df['volume']
del df['close_time']
del df['quote_asset_volume']
del df['number_of_trades']
del df['taker_buy_base_asset_volume']
del df['taker_buy_quote_asset_volume']


# Calculate oversell
df['oversell_day'] = 0
df.loc[df['high'] < df['ema_25'], 'oversell_day'] = 1
df.loc[df['ema_25'] == 0, 'oversell_day'] = 0


df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
df.loc[df['oversell_day'] == 0, 'streak'] = 0


# display
## swap columns ['time', 'high', 'low', 'open', 'close', 'ema_25']

df = df[['time', 'open', 'high', 'low', 'close', 'ema_25']]

print('Result')
df.tail(10)

,time,high,low,open,close,ema_25,oversell_day,streak
d_index,,,,,,,,
46,2022-10-12,0.00001170,0.00000857,0.00001125,0.00001097,0.00001622,1,22
47,2022-10-30,0.00001204,0.00000975,0.00001097,0.00001031,0.00001576,1,23
48,2022-11-17,0.00001132,0.00001009,0.00001032,0.00001096,0.00001539,1,24
49,2022-12-05,0.00001122,0.00000931,0.00001095,0.00000979,0.00001496,1,25
50,2022-12-23,0.00001010,0.00000880,0.00000978,0.00000993,0.00001457,1,26
51,2023-01-10,0.00001001,0.00000640,0.00000994,0.00000957,0.00001419,1,27
52,2023-01-28,0.00001147,0.00000860,0.00000958,0.00001000,0.00001387,1,28
53,2023-02-15,0.00001251,0.00000950,0.00001002,0.00000998,0.00001357,1,29
54,2023-03-05,0.00001024,0.00000790,0.00000997,0.00000805,0.00001314,1,30


Result


,time,open,high,low,close,ema_25
d_index,,,,,,
46,2022-10-12,0.00001125,0.00001170,0.00000857,0.00001097,0.00001622
47,2022-10-30,0.00001097,0.00001204,0.00000975,0.00001031,0.00001576
48,2022-11-17,0.00001032,0.00001132,0.00001009,0.00001096,0.00001539
49,2022-12-05,0.00001095,0.00001122,0.00000931,0.00000979,0.00001496
50,2022-12-23,0.00000978,0.00001010,0.00000880,0.00000993,0.00001457
51,2023-01-10,0.00000994,0.00001001,0.00000640,0.00000957,0.00001419
52,2023-01-28,0.00000958,0.00001147,0.00000860,0.00001000,0.00001387
53,2023-02-15,0.00001002,0.00001251,0.00000950,0.00000998,0.00001357
54,2023-03-05,0.00000997,0.00001024,0.00000790,0.00000805,0.00001314
